# Computational Hydrogen Electrode Tutorial

Import useful modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read
from ase import Atoms
from ase.build import mx2, molecule
from ase.constraints import FixAtoms
from ase.visualize import view
from ase.calculators.espresso import Espresso, EspressoProfile
import time

In [ ]:
MoS2 = mx2(formula='MoS2', kind='2H', a=3.18, thickness=3.19, size=(2, 2, 1), vacuum=10.)

In [ ]:
MoS2.positions

In [ ]:
# Optionally create profile to override paths in ASE configuration:
profile = EspressoProfile(
    command='mpirun -np 4 /Users/oliviero/PWSCF/espresso-git/bin/pw.x', pseudo_dir='./pseudos/'
)

In [ ]:
pseudopotentials = {
    "H":"H.pbe-rrkjus_psl.1.0.0.UPF",
    "O":"O.pbe-n-kjpaw_psl.0.1.UPF",
    "Mo":"Mo_ONCV_PBE-1.0.oncvpsp.upf",
    "S":"s_pbe_v1.4.uspp.F.UPF"
}

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2.calc = calc

In [ ]:
eMoS2 = MoS2.get_potential_energy()

In [ ]:
print(eMoS2)

In [ ]:
MoS2 = read('MoS2_vcrelax.out')

In [ ]:
MoS2.center()

In [ ]:
MoS2.positions

In [ ]:
atop_index = 1

In [ ]:
MoS2.positions[atop_index]

In [ ]:
fixed = list(range(len(MoS2)))
fixed.remove(atop_index)
print(fixed)
constraint = FixAtoms(indices=fixed)
MoS2.set_constraint(constraint)

In [ ]:
# Creating the OH molecule
oh_molecule = Atoms('OH', positions=[(0, 0, 0), (0, -0.763, 0.596)])

oh_molecule.translate(MoS2.positions[atop_index] + (0, 0, 1.87))

oh_molecule.positions

In [ ]:
MoS2OH = MoS2 + oh_molecule
MoS2OH.positions

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2OH'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2OH.calc = calc

In [ ]:
from ase.optimize import QuasiNewton

In [ ]:
dyn = QuasiNewton(MoS2OH, trajectory='MoS2OH.traj')
t = time.time()
dyn.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

In [ ]:
print("bond S-O: ", MoS2OH.get_distance(1,12))
print("bond O-H: ", MoS2OH.get_distance(12,13))

In [ ]:
# Creating the OH molecule
o_molecule = Atoms('O', positions=[(0, 0, 0)])

o_molecule.translate(MoS2.positions[atop_index] + (0, 0, 1.872))

o_molecule.positions

In [ ]:
MoS2O = MoS2 + o_molecule
MoS2O.positions

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2O'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2O.calc = calc

In [ ]:
dyn = QuasiNewton(MoS2O, trajectory='MoS2O.traj')
t = time.time()
dyn.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

In [ ]:
print("bond S-O: ", MoS2OH.get_distance(1,12))

In [ ]:
ooh_molecule = Atoms('OOH', positions=[(0, 0, 0), (0.75, 0.34, 1.10), (0.1, 0.25, 1.85)])

ooh_molecule.translate(MoS2.positions[atop_index] + (0, 0, 1.87))

ooh_molecule.positions

In [ ]:
MoS2OOH = MoS2 + ooh_molecule
MoS2OOH.positions

In [ ]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2O'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2OOH.calc = calc

In [ ]:
dyn = QuasiNewton(MoS2OOH, trajectory='MoS2OOH.traj')
t = time.time()
dyn.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

In [ ]:
h2_molecule = molecule("H2")
h2_molecule.set_cell(MoS2.get_cell())
h2_molecule.calc = calc
dynH2 = QuasiNewton(h2_molecule, trajectory='H2.traj')
t = time.time()
dynH2.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))
e_h2 = h2_molecule.get_potential_energy()
print("H2 energy: ", e_h2, " eV")
print("H-H bond lenght: ", h2_molecule.get_distance(0,1))

In [ ]:
h2o_molecule = molecule("H2O")
h2o_molecule.set_cell(MoS2.get_cell())
h2o_molecule.calc = calc
dynH2O = QuasiNewton(h2o_molecule, trajectory='H2O.traj')
t = time.time()
dynH2O.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))
e_h2o = h2o_molecule.get_potential_energy()
print("H2O energy: ", e_h2o, " eV")
print("O-H bond lenght: ", h2o_molecule.get_distance(0,1))

In [ ]:


"""
Calculations to get the free energy of the O intermediate adsorbed on the Ni surface
"""
o_molecule = Atoms('O', positions=[(0, 0, 0)])
slabNi = fcc111("Ni", a=lattice_constant_Ni, size=(2,2,2))
p = slabNi.positions[4]
o_molecule.translate(p + (0, 0, 1.5))
slabO = slabNi + o_molecule
slabO.center(vacuum=2.0)
constraint = FixAtoms(indices=[0,1,2,3,5,6,7])
slabO.set_constraint(constraint)
view(slabO)

slabO.set_calculator(calculator)
dynO = QuasiNewton(slabO, trajectory="O_Ni.traj", )
t = time.time()
dynO.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

e_slabO = slabO.get_potential_energy()
print("O on Ni(111) energy: ", e_slabO, " eV")
view(slabO)
print("bond Ni-O: ", slabO.get_distance(4,8))


"""
Calculations to get the free energy of the OOH intermediate adsorbed on the Ni surface
"""
ooh_molecule = Atoms('OOH', positions=[(0, 0, 0), (0, 0, 1.4), (0, -0.763, 2.0)])
slabNi = fcc111("Ni", a=lattice_constant_Ni, size=(2,2,2))
p = slabNi.positions[4]
ooh_molecule.translate(p + (0, 0, 1.5))
slabOOH = slabNi + ooh_molecule
slabOOH.center(vacuum=2.0)
constraint = FixAtoms(indices=[0,1,2,3,5,6,7])
slabOOH.set_constraint(constraint)
view(slabOOH)

slabOOH.set_calculator(calculator)
dynOOH = QuasiNewton(slabOOH, trajectory="OOH_Ni.traj", )
t = time.time()
dynOOH.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

e_slabO = slabOOH.get_potential_energy()
print("OOH on Ni(111) energy: ", e_slabO, " eV")
view(slabOOH)
print("bond Ni-O: ", slabOOH.get_distance(4,8))




"""
Calculations to get the free energy of H2O and H2 needed to calculate the free energies of the reaction
"""

calculator2 = GPAW(xc='RPBE', mode=PW(350), h=0.2, occupations=FermiDirac(0.1))


h2_molecule = molecule("H2")
h2_molecule.set_cell(slab.get_cell())
h2_molecule.center(vacuum=2.0)
h2_molecule.set_calculator(calculator2)
dynH2 = QuasiNewton(h2_molecule, trajectory='H2.traj')
t = time.time()
dynH2.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))
e_h2 = h2_molecule.get_potential_energy()
print("H2 energy: ", e_h2, " eV")
print("H-H bond lenght: ", h2_molecule.get_distance(0,1))